## Training pipeline

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.keyvault.secrets import SecretClient

### Configure key-vault to get secrets

In [2]:
keyVaultName = "mobilepricinga6945442583"
KVUri = f"https://{keyVaultName}.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

### Authenticate

In [3]:
credential = DefaultAzureCredential()

### Get Secrets from key-vault

In [4]:
subs_id = client.get_secret("subscription-id").value
rg_name = client.get_secret("ml-resource-group").value
ws_name = client.get_secret("ml-workspace-name").value

### Create Client

In [5]:
ml_client = MLClient(
    credential=credential,
    subscription_id=subs_id,
    resource_group_name=rg_name,
    workspace_name=ws_name,
)

### Get Data Asset

In [6]:
data_asset = ml_client.data.get("mobile_pricing_train_data_v1", version="1")

### Load Components

In [7]:
data_prep_component = ml_client.components.get("data_prep_mobile_pricing")

In [8]:
model_training_component =ml_client.components.get("model_training_mobile_pricing")

### Create Pipeline

In [9]:
from azure.ai.ml import dsl, Input, Output

In [10]:
@dsl.pipeline(compute="serverless",description="Mobile Pricing Pipeline")
def create_pipeline(pipeline_input_data,model_name):
    data_prep_job = data_prep_component(train_data = pipeline_input_data)
    model_train_job = model_training_component(train = data_prep_job.outputs.train,
                                               test = data_prep_job.outputs.test,
                                              model_name=model_name)
    res = {"pipeline_job_train":data_prep_job.outputs.train,
          "pipeline_job_test":data_prep_job.outputs.test}
    return res

In [11]:
model_name = "mobile_pricing_model"
pipeline = create_pipeline(pipeline_input_data=Input(path=data_asset.path,type="uri_file"),model_name=model_name)

In [12]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="Mobile pricing",
)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [13]:
ml_client.jobs.stream(pipeline_job.name)

RunId: sincere_receipt_sx9yrn82pc
Web View: https://ml.azure.com/runs/sincere_receipt_sx9yrn82pc?wsid=/subscriptions/0aa1c63a-7a46-403c-91e4-8ec91264bc42/resourcegroups/rg-mobile-pricing-dev/workspaces/mobile-pricing-azml-dev

Streaming logs/azureml/executionlogs.txt

[2024-02-28 07:39:07Z] Submitting 1 runs, first five are: afbf0205:8e9d25fc-fa43-4140-a34e-33eab99077af
[2024-02-28 07:45:01Z] Completing processing run id 8e9d25fc-fa43-4140-a34e-33eab99077af.
[2024-02-28 07:45:01Z] Submitting 1 runs, first five are: 15639a0d:78cb94e5-27fb-4ee1-a13b-a3c72f6b5c79
[2024-02-28 07:46:09Z] Completing processing run id 78cb94e5-27fb-4ee1-a13b-a3c72f6b5c79.

Execution Summary
RunId: sincere_receipt_sx9yrn82pc
Web View: https://ml.azure.com/runs/sincere_receipt_sx9yrn82pc?wsid=/subscriptions/0aa1c63a-7a46-403c-91e4-8ec91264bc42/resourcegroups/rg-mobile-pricing-dev/workspaces/mobile-pricing-azml-dev

